In [43]:
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from stanfordcorenlp import StanfordCoreNLP
import json

# Для запуска необходимо развернуть на локальной машине Stanford CoreNLP
# https://nlp.stanford.edu/software/stanford-corenlp-latest.zip

# Команда для запуска:
# java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -annotators "tokenize,ssplit,pos,lemma,parse,sentiment" -port 9000 -timeout 30000
nlp = StanfordCoreNLP('http://localhost', port=9000, timeout=30000)
props = {'annotators': 'pos,lemma',
         'pipelineLanguage': 'en',
         'outputFormat': 'json'}
stop_words = stopwords.words('english')

In [44]:
def tokenize(text):
    lemmas = {}
    words = []
    # Разделяем полученный текст на предложения
    sentences = sent_tokenize(text)
    for sentence in sentences:
        # Разделяем предложение на отдельные слова-токены
        tokens = word_tokenize(sentence)
        # Очищаем предложение от стоп-слов и символов пунктуации
        cleared_sentence = ' '.join([word.lower() for word in tokens if word.isalpha() and word not in stop_words])
        # Для пустых строк нет необходимости вычислять леммы
        if not cleared_sentence.strip():
            continue
        # С помощью Stanford CoreNLP вычисляем параметры предложения в виде JSON
        parsed_str = nlp.annotate(cleared_sentence, properties=props)
        parsed_dict = json.loads(parsed_str)
        for word_data in parsed_dict['sentences'][0]['tokens']:
            # Получаем оригинальное слово из предложения
            sentence_word = word_data['originalText'].lower()
            # Получаем лемму слова
            word_lemma = word_data['lemma']
            # Заполняем список токенов/словарь лемм
            words.append(sentence_word)
            if word_lemma in lemmas:
                lemmas[word_lemma] = lemmas[word_lemma] + [sentence_word]
            else:
                lemmas[word_lemma] = [sentence_word]
    return words, lemmas

In [45]:
# Метод для склеивания двух словарей
def merge_dicts(dict1, dict2):
    for key in dict2:
        if key in dict1:
            dict1[key] = dict1[key] + dict2[key]
        else:
            dict1[key] = dict2[key]
    return dict1

In [46]:
# Метод записи лемм в файл
def write_lemmas_to_file(file_path, lemmas_dict):
    with open(file_path, 'w') as lemmas_file:
        for key in lemmas_dict:
            lemma_str = key + ': ' + ' '.join([val for val in set(lemmas_dict[key])])
            lemmas_file.write(lemma_str + '\n')

In [47]:
# Метод записи токенов в файл
def write_tokens_to_file(file_path, tokens):
    with open(file_path, 'w') as tokens_file:
        tokens_str = '\n'.join([val for val in tokens])
        tokens_file.write(tokens_str)

In [48]:
import os

data_dir_path = '../Выкачка/'
data_file_pattern = 'выкачка'
lemmas = {}
tokens = []

# Для каждого файла выкачки собираем токены и леммы и склеиваем в общий список для записи в файлы
for filename in os.listdir(data_dir_path):
    if data_file_pattern not in filename:
        continue
    file_path = data_dir_path + filename
    with open(file_path, 'r') as text_file:
        file_tokens, file_lemmas = tokenize(text_file.read())
        lemmas = merge_dicts(lemmas, file_lemmas)
        tokens = tokens + file_tokens
        
write_tokens_to_file('../tokens.txt', tokens)
write_lemmas_to_file('../lemmas.txt', lemmas)